In [ ]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 32.1 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="hkTkjFCZ5Ul7fGujCKmA")
project = rf.workspace("intellifone").project("damage-detection-d0coe")
version = project.version(5)
dataset = version.download("yolov11")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.9/49.9 MB 24.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 87.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 128.6 MB/s eta 0:00:00
  Attempting uninstall: opencv-python-headless
    Found existing installation: opencv-python-headless 4.12.0.88
    Uninstalling opencv-python-headless-4.12.0.88:
      Successfully uninstalled opencv-python-headless-4.12.0.88
  Attempting uninstall: idna
    Found existing installation: idna 3.10
    Uninstalling idna-3.10:
      Successfully uninstalled idna-3.10
loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to Damage-detection-5 in yolov11:: 100%|██████████| 4206/4206 [00:00<00:00, 7548.76it/s]


In [ ]:
import os, yaml, glob
from collections import Counter

dataset_dir = "/content/Damage-detection-5"
keep_classes = ['crack', 'dot', 'line', 'scratch']

# --- Load YAML ---
data_yaml = os.path.join(dataset_dir, "data.yaml")
with open(data_yaml, "r") as f:
    data = yaml.safe_load(f)

class_names = data["names"]
keep_ids = [class_names.index(c) for c in keep_classes if c in class_names]
print(f"📘 Found classes in dataset: {class_names}")
print(f"✅ Keeping class IDs: {keep_ids} → {keep_classes}")

# --- Filter label files ---
for split in ["train", "valid", "test"]:
    label_dir = os.path.join(dataset_dir, split, "labels")
    if not os.path.exists(label_dir):
        continue

    for file in os.listdir(label_dir):
        path = os.path.join(label_dir, file)
        with open(path, "r") as f:
            lines = f.readlines()

        new_lines = []
        for line in lines:
            parts = line.strip().split()
            cls_id = int(parts[0])
            if cls_id in keep_ids:
                new_cls = keep_ids.index(cls_id)
                parts[0] = str(new_cls)
                new_lines.append(" ".join(parts) + "\n")

        if new_lines:
            with open(path, "w") as f:
                f.writelines(new_lines)
        else:
            os.remove(path)

# --- Update YAML ---
data["names"] = keep_classes
data["nc"] = len(keep_classes)
with open(data_yaml, "w") as f:
    yaml.safe_dump(data, f)

print("\n✅ Segmentation dataset filtered to:", keep_classes)

# --- Count label occurrences ---
counts = Counter()
for split in ["train", "valid", "test"]:
    label_dir = os.path.join(dataset_dir, split, "labels")
    if not os.path.exists(label_dir):
        continue
    for label_file in glob.glob(f"{label_dir}/*.txt"):
        with open(label_file, "r") as f:
            for line in f:
                cls_id = int(line.split()[0])
                counts[cls_id] += 1

print("\n📊 Label count after filtering:")
for i, name in enumerate(keep_classes):
    print(f"  {i} ({name}): {counts.get(i, 0)} instances")
print(f"\n📦 Total labeled objects: {sum(counts.values())}")

📘 Found classes in dataset: ['crack', 'dot', 'line', 'scratch']
✅ Keeping class IDs: [0, 1, 2, 3] → ['crack', 'dot', 'line', 'scratch']

✅ Segmentation dataset filtered to: ['crack', 'dot', 'line', 'scratch']

📊 Label count after filtering:
  0 (crack): 1296 instances
  1 (dot): 967 instances
  2 (line): 1261 instances
  3 (scratch): 820 instances

📦 Total labeled objects: 4344


In [ ]:
!yolo train \
    model=/content/drive/MyDrive/yolo_runs/damage_seg_crack_dot_line/weights/best.pt \
    data=/content/Damage-detection-5/data.yaml \
    epochs=50 \
    imgsz=640 \
    batch=16 \
    device=0 \
    project="/content/drive/MyDrive/yolo_runs" \
    name="damage_seg_crack_dot_line_scratch" \
    cache=True \
    workers=4


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
Ultralytics 8.3.214 🚀 Python-3.12.12 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=True, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/Damage-detection-5/data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, ke

In [ ]:
!yolo val \
    model="/content/drive/MyDrive/yolo_runs/damage_seg_crack_dot_line_scratch/weights/best.pt" \
    data=/content/Damage-detection-5/data.yaml \
    split=test

Ultralytics 8.3.214 🚀 Python-3.12.12 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
YOLO11s-seg summary (fused): 113 layers, 10,068,364 parameters, 0 gradients, 32.8 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 15.5±4.7 MB/s, size: 30.1 KB)
val: Scanning /content/Damage-detection-5/test/labels... 112 images, 9 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 121/121 705.8it/s 0.2s
val: New cache created: /content/Damage-detection-5/test/labels.cache
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 8/8 1.9it/s 4.3s
                   all        121        249      0.499      0.437      0.473      0.302      0.464      0.404      0.417      0.218
                 crack         28         61      0.335      0.231      0.249      0.177      0.287      0.197      0.176      0.106
                   dot         45         54       0.87      0.745      0.819      0.501      0.845      0.

In [ ]:
!yolo val \
    model=/content/drive/MyDrive/yolo_runs/damage_seg_crack_dot_line_scratch/weights/best.pt \
    data=/content/Damage-detection-5/data.yaml \
    task=detect \
    device=0


WARNING ⚠️ conflicting 'task=detect' passed with 'task=segment' model. Ignoring 'task=detect' and updating to 'task=segment' to match model.
Ultralytics 8.3.214 🚀 Python-3.12.12 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
YOLO11s-seg summary (fused): 113 layers, 10,068,364 parameters, 0 gradients, 32.8 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 464.3±615.3 MB/s, size: 41.9 KB)
val: Scanning /content/Damage-detection-5/valid/labels.cache... 226 images, 16 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 242/242 4.4Mit/s 0.0s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 16/16 2.2it/s 7.3s
                   all        242        474      0.577       0.43       0.47       0.31      0.552      0.393      0.413      0.219
                 crack         64        145      0.535       0.29      0.319       0.21       0.48      0.241      0.235      0.149
                   dot  

In [ ]:
!yolo predict \
    model="/content/drive/MyDrive/yolo_runs/damage_seg_crack_dot_line/weights/best.pt" \
    source="/content/mypic.jpeg" \
    save=True \
    save_txt=True

Ultralytics 8.3.207 🚀 Python-3.12.11 torch-2.8.0+cu126 CPU (Intel Xeon CPU @ 2.20GHz)
YOLO11s-seg summary (fused): 113 layers, 10,067,977 parameters, 0 gradients, 32.8 GFLOPs

image 1/1 /content/mypic.jpeg: 640x384 3 lines, 491.3ms
Speed: 3.7ms preprocess, 491.3ms inference, 11.1ms postprocess per image at shape (1, 3, 640, 384)
Results saved to /content/runs/segment/predict5
1 label saved to /content/runs/segment/predict5/labels
💡 Learn more at https://docs.ultralytics.com/modes/predict


In [ ]:
from ultralytics import YOLO

model = YOLO("/content/drive/MyDrive/yolo_runs/damage_seg_crack_dot_line/weights/best.pt")
results = model.predict("/content/mypic.jpeg")

print(results[0].orig_shape)   # (H, W) of original image
print(results[0].masks.data.shape if results[0].masks else "No masks")



image 1/1 /content/mypic.jpeg: 640x384 3 lines, 734.5ms
Speed: 14.8ms preprocess, 734.5ms inference, 40.3ms postprocess per image at shape (1, 3, 640, 384)
(1280, 720)
torch.Size([3, 640, 384])
